# Chapter 25 - Preprocessing - Bucketing

In [8]:
%ShowTypes on

Types will be printed.


# Setup

In [34]:
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Row

import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.DataFrame

import org.apache.spark.ml.feature._
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector

import spark.implicits._

import java.time.temporal.ChronoUnit
import java.time.{Period, LocalDate, Instant}
import java.sql.Timestamp

In [10]:
%%html
<!-- To left align the HTML components in Markdown -->
<style>
table {float:left}
</style>

<!-- To left align the HTML components in Markdown -->

### Spark parition control based on core availability

In [11]:
val NUM_CORES = 4
val NUM_PARTITIONS = 4

lazy val spark: SparkSession = SparkSession.builder()
    .appName("mllib-cross-validation")
    .getOrCreate()

spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
/*
spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.driver.memory", "6g")
spark.conf.set("spark.executor.memory", "2g")
spark.conf.set("spark.master", "spark://masa:7077")
*/

NUM_CORES: Int = 4
NUM_PARTITIONS: Int = 4
spark: org.apache.spark.sql.SparkSession = <lazy>


spark: org.apache.spark.sql.SparkSession = <lazy>


In [12]:
val configMap = spark.conf.getAll.foreach(println)

(spark.serializer,org.apache.spark.serializer.KryoSerializer)
(spark.driver.host,172.17.0.1)
(spark.eventLog.enabled,true)
(spark.driver.port,35271)
(spark.hadoop.validateOutputSpecs,True)
(spark.repl.class.uri,spark://172.17.0.1:35271/classes)
(spark.jars,file:/home/oonisim/.local/share/jupyter/kernels/apache_toree_scala/lib/toree-assembly-0.3.0-incubating.jar)
(spark.repl.class.outputDir,/tmp/spark-0457f37e-a701-4ec8-a1fb-241b65acf6f1/repl-33d143d8-b21e-4a5b-9877-5ed0f16b42e6)
(spark.app.name,mllib-cross-validation)
(spark.driver.memory,3g)
(spark.executor.instances,2)
(spark.history.fs.logdirectory,hdfs://oonisim:8020/logs_spark)
(spark.default.parallelism,16)
(spark.executor.id,driver)
(spark.submit.deployMode,client)
(spark.master,yarn)
(spark.ui.filters,org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter)
(spark.executor.memory,4g)
(spark.eventLog.dir,hdfs://oonisim:8020/logs_spark)
(spark.executor.cores,4)
(spark.driver.appUIAddress,http://172.17.0.1:4040)
(spark.org.apac

configMap: Unit = ()


## Constants

In [13]:
val PROTOCOL="file://"
val DATA_DIR="/home/oonisim/home/repositories/git/oonisim/spark-programs/Dataframe/data"
val RESULT_DIR="."

PROTOCOL: String = file://
DATA_DIR: String = /home/oonisim/home/repositories/git/oonisim/spark-programs/Dataframe/data
RESULT_DIR: String = .


RESULT_DIR: String = .


# Dataframe

In [16]:
val sales = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load(PROTOCOL + DATA_DIR + "/retail-data/by-day/*.csv")
  .coalesce(5)
  .where("Description IS NOT NULL")
val fakeIntDF = spark.read.parquet(PROTOCOL + DATA_DIR + "/simple-ml-integers")
var simpleDF = spark.read.json(PROTOCOL + DATA_DIR + "/simple-ml")
val scaleDF = spark.read.parquet(PROTOCOL + DATA_DIR + "/simple-ml-scaling")

sales: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [InvoiceNo: string, StockCode: string ... 6 more fields]
fakeIntDF: org.apache.spark.sql.DataFrame = [int1: int, int2: int ... 1 more field]
simpleDF: org.apache.spark.sql.DataFrame = [color: string, lab: string ... 2 more fields]
scaleDF: org.apache.spark.sql.DataFrame = [id: int, features: vector]


lastException: Throwable = null


scaleDF: org.apache.spark.sql.DataFrame = [id: int, features: vector]


In [18]:
sales.cache()
sales.printSchema
sales.show()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538| 

In [59]:
val contDF = spark.range(20).selectExpr("cast(id as double)")
contDF.show(5)

+---+
| id|
+---+
|0.0|
|1.0|
|2.0|
|3.0|
|4.0|
+---+
only showing top 5 rows



contDF: org.apache.spark.sql.DataFrame = [id: double]


contDF: org.apache.spark.sql.DataFrame = [id: double]


# Bucketing

In [58]:
import org.apache.spark.ml.feature.Bucketizer
val bucketBorders = Array(-1.0, 5.0, 10.0, 250.0, 600.0)
val bucketer = new Bucketizer().setSplits(bucketBorders).setInputCol("id")
bucketer.transform(contDF).show()

+----+-------------------------------+
|  id|bucketizer_80c1543d48e8__output|
+----+-------------------------------+
| 0.0|                            0.0|
| 1.0|                            0.0|
| 2.0|                            0.0|
| 3.0|                            0.0|
| 4.0|                            0.0|
| 5.0|                            1.0|
| 6.0|                            1.0|
| 7.0|                            1.0|
| 8.0|                            1.0|
| 9.0|                            1.0|
|10.0|                            2.0|
|11.0|                            2.0|
|12.0|                            2.0|
|13.0|                            2.0|
|14.0|                            2.0|
|15.0|                            2.0|
|16.0|                            2.0|
|17.0|                            2.0|
|18.0|                            2.0|
|19.0|                            2.0|
+----+-------------------------------+



bucketBorders: Array[Double] = Array(-1.0, 5.0, 10.0, 250.0, 600.0)
bucketer: org.apache.spark.ml.feature.Bucketizer = bucketizer_80c1543d48e8


bucketer: org.apache.spark.ml.feature.Bucketizer = bucketizer_80c1543d48e8


## Percentile Bucketing

In [62]:
import org.apache.spark.ml.feature.QuantileDiscretizer
val bucketer = new QuantileDiscretizer().setNumBuckets(5).setInputCol("id").setOutputCol("bucket")
val fittedBucketer = bucketer.fit(contDF)
fittedBucketer.transform(contDF).show()

+----+------+
|  id|bucket|
+----+------+
| 0.0|   0.0|
| 1.0|   0.0|
| 2.0|   0.0|
| 3.0|   1.0|
| 4.0|   1.0|
| 5.0|   1.0|
| 6.0|   1.0|
| 7.0|   2.0|
| 8.0|   2.0|
| 9.0|   2.0|
|10.0|   2.0|
|11.0|   2.0|
|12.0|   3.0|
|13.0|   3.0|
|14.0|   3.0|
|15.0|   4.0|
|16.0|   4.0|
|17.0|   4.0|
|18.0|   4.0|
|19.0|   4.0|
+----+------+



bucketer: org.apache.spark.ml.feature.QuantileDiscretizer = quantileDiscretizer_631e9b33936d
fittedBucketer: org.apache.spark.ml.feature.Bucketizer = quantileDiscretizer_631e9b33936d


fittedBucketer: org.apache.spark.ml.feature.Bucketizer = quantileDiscretizer_631e9b33936d
